In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample


warnings.filterwarnings("ignore")

#import pingouin as pg
from sklearn.utils import resample

In [2]:
from statannot import add_stat_annotation
import seaborn as sns


In [3]:
import numpy as np
from scipy.stats import percentileofscore
from sklearn.utils import check_random_state
from itertools import islice

def permtest(x, y, statistic="mean", max_samples=5000, random_state=42):
    """
    Conducts a permutation test between two independent samples.
    Parameters
    ----------
    x : ndarray
        First set of datapoints.
    y : ndarray
        Second set of datapoints.
    statistic : str or callable, optional
        Function that takes in samples x and y and reports
        statistic of interest. By default, "mean" reports
        the difference of sample means. List of default
        options are ("mean", "median").
    max_samples : int, optional.
        Maximum number of label permutations to try.
    random_state : np.random.RandomState, int, or None.
        If specified, used to seed the random number generator to shuffle the ordering of the datapoints.
    """

    # Initialize random state and function of interest
    rs = check_random_state(random_state)
    stat_func = _get_stat_func(statistic)

    # Concatenate samples
    xy = np.concatenate((x, y))
    n_x = x.size

    # Observed statistic
    observed_stat = stat_func(x, y)

    # Set the number of permutations to compute
    n_perms = max(max_samples, 100000)  # Limit the number of permutations to a manageable number (e.g., 10000)

    # Allocate space for computed statistics
    shuffled_stats = np.empty(n_perms)

    # Print coverage
    print(f"Computing permutations...")

    # Sampling random permutations
    for i in range(n_perms):
        rs.shuffle(xy)
        x_ = xy[:n_x]
        y_ = xy[n_x:]
        shuffled_stats[i] = stat_func(x_, y_)

    # Compute a two-sided p-value. We take the smallest
    # percentile and then multiply by two.
    pval = 2 * min(
        percentileofscore(shuffled_stats, observed_stat, kind="rank") / 100,
        1 - percentileofscore(shuffled_stats, observed_stat, kind="rank") / 100
    )

    return pval

def _get_stat_func(name_or_func):
    """
    Instantiates functions that compute default statistics of interest.
    """
    # If specified function is callable
    if not isinstance(name_or_func, str):
        if callable(name_or_func):
            return name_or_func
        else:
            raise ValueError(
                "`statistic` should be a string like ('mean', 'median')"
                " or a function that takes in samples x, y and returns"
                " the statistic of interest."
            )

    # Default functions
    if name_or_func == "mean":
        return lambda x, y: np.mean(x) - np.mean(y)
    elif name_or_func == "median":
        return lambda x, y: np.median(x) - np.median(y)
    else:
        raise ValueError("Did not recognize statistic.")



In [4]:
import numpy as np

def pooled(x, y, func):
    nx = len(x)
    ny = len(y)
    return np.sqrt(((nx - 1) * func(x) ** 2 + (ny - 1) * func(y) ** 2) / (nx + ny - 2))

def hdmedian(x):
    return np.median(x)

def hdmad(x):
    return 1.4826 * hdmedian(np.abs(x - hdmedian(x)))

def phdmad(x, y):
    return pooled(x, y, hdmad)

def gamma_effect_size(x, y, prob):
    hdquantile_x = np.percentile(x, prob * 100)
    hdquantile_y = np.percentile(y, prob * 100)
    return (hdquantile_y - hdquantile_x) / phdmad(x, y)

x = np.array([1, 2, 3, 4, 5])
y = np.array([3, 4, 5, 6, 7])

prob = 0.5  
gamma_effect = gamma_effect_size(x, y, prob)
print(f'Tamaño del efecto gamma: {gamma_effect}')


Tamaño del efecto gamma: 1.3489815189531904


In [5]:
def remove_outliers_iqr_01(data):

    data = np.array(data)

    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1

    ir_mult = 0.1
    lower_bound = Q1 - ir_mult * IQR
    upper_bound = Q3 + ir_mult * IQR

    filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]
    
    return filtered_data

def remove_outliers_iqr_05(data):

    data = np.array(data)

    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1

    ir_mult = 0.5
    lower_bound = Q1 - ir_mult * IQR
    upper_bound = Q3 + ir_mult * IQR

    filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]
    
    return filtered_data

def remove_outliers_iqr_10(data):

    data = np.array(data)

    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1

    ir_mult = 1.0
    lower_bound = Q1 - ir_mult * IQR
    upper_bound = Q3 + ir_mult * IQR

    filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]
    
    return filtered_data

def cliff_delta(group1, group2):

    group1_ = remove_outliers_iqr_01(group1)
    group2_ = remove_outliers_iqr_01(group2)
    
    sub_sample_size = np.min([int(group1_.shape[0]*0.7), int(group2_.shape[0]*0.7)])

    delta_l = []
    for i in range(10):
       
        group1 = resample(group1_, n_samples=sub_sample_size, replace=False, random_state=i)
        group2 = resample(group2_, n_samples=sub_sample_size, replace=False, random_state=i)
    
        group1_exp = group1[:, np.newaxis]  
        group2_exp = group2[np.newaxis, :]  
    
        greater = np.sum(group1_exp > group2_exp)
        less = np.sum(group1_exp < group2_exp)
    
        n1, n2 = len(group1), len(group2)
        delta = (greater - less) / (n1 * n2)

    delta_l.append(delta)

    print('N', n1+n2, n1, n2)

    return (np.round(np.mean(delta_l), 4), np.round(np.std(delta_l)*2.58,100))

def cliff_delta05(group1, group2):

    group1_ = remove_outliers_iqr_05(group1)
    group2_ = remove_outliers_iqr_05(group2)
    
    sub_sample_size = np.min([int(group1_.shape[0]*0.7), int(group2_.shape[0]*0.7)])


    delta_l = []
    for i in range(10):
       
        group1 = resample(group1_, n_samples=sub_sample_size, replace=False, random_state=i)
        group2 = resample(group2_, n_samples=sub_sample_size, replace=False, random_state=i)
    
        group1_exp = group1[:, np.newaxis]  
        group2_exp = group2[np.newaxis, :]   
    
        greater = np.sum(group1_exp > group2_exp)
        less = np.sum(group1_exp < group2_exp)
    
        n1, n2 = len(group1), len(group2)
        delta = (greater - less) / (n1 * n2)

    delta_l.append(delta)

    print('N', n1+n2, n1, n2)

    return (np.round(np.mean(delta_l), 4), np.round(np.std(delta_l)*2.58,100))

def cliff_delta10(group1, group2):

    group1_ = remove_outliers_iqr_10(group1)
    group2_ = remove_outliers_iqr_10(group2)
    
    sub_sample_size = np.min([int(group1_.shape[0]*0.7), int(group2_.shape[0]*0.7)])


    delta_l = []
    for i in range(10):
       
        group1 = resample(group1_, n_samples=sub_sample_size, replace=False, random_state=i)
        group2 = resample(group2_, n_samples=sub_sample_size, replace=False, random_state=i)
    
        group1_exp = group1[:, np.newaxis]  
        group2_exp = group2[np.newaxis, :]  
    
        greater = np.sum(group1_exp > group2_exp)
        less = np.sum(group1_exp < group2_exp)
    
        n1, n2 = len(group1), len(group2)
        delta = (greater - less) / (n1 * n2)

    delta_l.append(delta)

    print('N', n1+n2, n1, n2)

    return (np.round(np.mean(delta_l), 4), np.round(np.std(delta_l)*2.58,100))


def calculate_z_from_u(group1, group2, u_statistic):
    group1 = remove_outliers_iqr(group1)
    group2 = remove_outliers_iqr(group2)

    n1, n2 = len(group1), len(group2)

    mu_u = (n1 * n2) / 2
    sigma_u = np.sqrt((n1 * n2 * (n1 + n2 + 1)) / 12)

    z_value = (u_statistic - mu_u) / sigma_u

    r_value = z_value / np.sqrt(n1 + n2)

    return r_value

def rank_biserial_from_u(group1, group2):
    group1 = remove_outliers_iqr_01(group1)
    group2 = remove_outliers_iqr_01(group2)

    u_statistic, _ = stats.mannwhitneyu(group1, group2, alternative='two-sided')

    n1, n2 = len(group1), len(group2)

    r_biserial = 1 - (2 * u_statistic) / (n1 * n2)

    return r_biserial

def rank_biserial_from_u10(group1, group2):
    group1 = remove_outliers_iqr_10(group1)
    group2 = remove_outliers_iqr_10(group2)

    u_statistic, _ = stats.mannwhitneyu(group1, group2, alternative='two-sided')

    n1, n2 = len(group1), len(group2)

    r_biserial = 1 - (2 * u_statistic) / (n1 * n2)

    return r_biserial

In [6]:
BAGs_data = pd.read_excel('data/BAGs_for_statistical_test.xlsx')

In [7]:
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt


order = ['Europe', 'Asia', 'LatinAmerica']

regions = BAGs_data['Region1'].unique()
mann_whitney_results = []

for i in range(len(regions)):
    for j in range(i + 1, len(regions)):
        region1_data = BAGs_data[BAGs_data['Region1'] == regions[i]]['GAP_corrected']
        region2_data = BAGs_data[BAGs_data['Region1'] == regions[j]]['GAP_corrected']


        sub_sample_size = np.min([int(region1_data.shape[0]*0.9), int(region2_data.shape[0]*0.9)])
        region1_data = resample(region1_data, n_samples=sub_sample_size, replace=False, random_state=42)
        region2_data = resample(region2_data, n_samples=sub_sample_size, replace=False, random_state=42)

        stat, p_value = stats.mannwhitneyu(region1_data, region2_data, alternative='two-sided')
        
        cliff_d, cliff_ci = cliff_delta(np.array(region1_data), np.array(region2_data))
        print( "cliff delta",cliff_d, cliff_ci)
        
        cliff_d05, cliff_ci05 = cliff_delta05(np.array(region1_data), np.array(region2_data))
        cliff_d10, cliff_ci10 = cliff_delta10(np.array(region1_data), np.array(region2_data))

        r_b = rank_biserial_from_u(np.array(region1_data), np.array(region2_data))
        r_b10 = rank_biserial_from_u10(np.array(region1_data), np.array(region2_data))

        print(f"No parametric cliff_delta: {cliff_d:.3f}")

        p_value_ = permtest(region1_data, region2_data)
        mann_whitney_results.append((regions[i], regions[j], cliff_d, cliff_ci, cliff_d05, cliff_ci05, cliff_d10, r_b10, cliff_ci10, p_value_*len(regions), p_value*len(regions)))


results_list = []
for result in mann_whitney_results:

    result_dict = {
        "Group1": result[0],
        "Group2": result[1],
        "cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[2]),
        "CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[3]),
        "cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[4]),
        "CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[5]),
        "cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[6]),
        "CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[7]),
        "p-value 100000 perm": np.abs(result[8]),
        "p-value mann whitney": np.abs(result[9]),
    }
    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)
display(results_df)


N 11394 5697 5697
cliff delta -0.0149 0.0
N 15558 7779 7779
N 18274 9137 9137
No parametric cliff_delta: -0.015
Computing permutations...
N 11564 5782 5782
cliff delta 0.3191 0.0
N 16344 8172 8172
N 18932 9466 9466
No parametric cliff_delta: 0.319
Computing permutations...
N 29364 14682 14682
cliff delta 0.4306 0.0
N 39840 19920 19920
N 46754 23377 23377
No parametric cliff_delta: 0.431
Computing permutations...


,Group1,Group2,cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L),CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L),p-value 100000 perm,p-value mann whitney
0,Asia,LatinAmerica,0.0149,0.0,0.0079,0.0,0.0001,0.003465,0.0,1.06368
1,Asia,Europa,0.3191,0.0,0.2359,0.0,0.1843,0.180764,0.0,0.00000
2,LatinAmerica,Europa,0.4306,0.0,0.3141,0.0,0.2333,0.234525,0.0,0.00000


In [8]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt


BAGs_data_europe = BAGs_data[BAGs_data['Region1'] == 'Europa']

order = ['WesternEurope', 'EasternEurope', 'NorthernEurope', 'SouthernEurope']


BAGs_data_no_outliers = BAGs_data_europe

order = ['WesternEurope', 'EasternEurope', 'NorthernEurope', 'SouthernEurope']


regions = BAGs_data_no_outliers['Region'].unique()
mann_whitney_results = []

display(BAGs_data_europe['Region'].value_counts())

for i in range(len(regions)):
    for j in range(i + 1, len(regions)):
        region1_data = BAGs_data_no_outliers[BAGs_data_no_outliers['Region'] == regions[i]]['GAP_corrected']
        region2_data = BAGs_data_no_outliers[BAGs_data_no_outliers['Region'] == regions[j]]['GAP_corrected']


        sub_sample_size = np.min([int(region1_data.shape[0]*0.9), int(region2_data.shape[0]*0.9)])
        region1_data = resample(region1_data, n_samples=sub_sample_size, replace=False, random_state=42)
        region2_data = resample(region2_data, n_samples=sub_sample_size, replace=False, random_state=42)
        
        stat, p_value = stats.mannwhitneyu(region1_data, region2_data, alternative='two-sided')
        
        cliff_d, cliff_ci = cliff_delta(np.array(region1_data), np.array(region2_data))
        print( "cliff delta",cliff_d, cliff_ci)
        
        cliff_d05, cliff_ci05 = cliff_delta05(np.array(region1_data), np.array(region2_data))
        cliff_d10, cliff_ci10 = cliff_delta10(np.array(region1_data), np.array(region2_data))

        r_b = rank_biserial_from_u(np.array(region1_data), np.array(region2_data))
        r_b10 = rank_biserial_from_u10(np.array(region1_data), np.array(region2_data))

        print(f"No parametric cliff_delta: {cliff_d:.3f}")
        print(f"No parametric biseral: {r_b:.3f}")

        p_value_ = permtest(region1_data, region2_data)
        mann_whitney_results.append((regions[i], regions[j], cliff_d, cliff_ci, cliff_d05, cliff_ci05, cliff_d10, r_b10, cliff_ci10, p_value_*len(regions), p_value*len(regions)))


results_list = []
for result in mann_whitney_results:

    result_dict = {
        "Group1": result[0],
        "Group2": result[1],
        "cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[2]),
        "CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[3]),
        "cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[4]),
        "CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[5]),
        "cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[6]),
        "CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[7]),
        "p-value 100000 perm": np.abs(result[8]),
        "p-value mann whitney": np.abs(result[9]),
    }
    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)
display(results_df)


Region
EasternEurope     13286
WesternEurope     10317
NorthernEurope     9334
SouthernEurope     7729
Name: count, dtype: int64

N 7486 3743 3743
cliff delta -0.1547 0.0
N 10560 5280 5280
N 12322 6161 6161
No parametric cliff_delta: -0.155
No parametric biseral: 0.145
Computing permutations...
N 6826 3413 3413
cliff delta -0.1024 0.0
N 9598 4799 4799
N 11174 5587 5587
No parametric cliff_delta: -0.102
No parametric biseral: 0.102
Computing permutations...
N 5646 2823 2823
cliff delta -0.1749 0.0
N 7888 3944 3944
N 9166 4583 4583
No parametric cliff_delta: -0.175
No parametric biseral: 0.198
Computing permutations...
N 6762 3381 3381
cliff delta 0.0491 0.0
N 9534 4767 4767
N 11152 5576 5576
No parametric cliff_delta: 0.049
No parametric biseral: -0.046
Computing permutations...
N 5594 2797 2797
cliff delta -0.0216 0.0
N 7888 3944 3944
N 9166 4583 4583
No parametric cliff_delta: -0.022
No parametric biseral: 0.033
Computing permutations...
N 5646 2823 2823
cliff delta -0.0774 0.0
N 7888 3944 3944
N 9166 4583 4583
No parametric cliff_delta: -0.077
No parametric biseral: 0.094
Computing permutations...


,Group1,Group2,cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L),CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L),p-value 100000 perm,p-value mann whitney
0,WesternEurope,EasternEurope,0.1547,0.0,0.0930,0.0,0.0734,0.073326,0.0,0.00000
1,WesternEurope,NorthernEurope,0.1024,0.0,0.0628,0.0,0.0434,0.044970,0.0,0.00000
2,WesternEurope,SouthernEurope,0.1749,0.0,0.1266,0.0,0.1054,0.099198,0.0,0.00000
3,EasternEurope,NorthernEurope,0.0491,0.0,0.0416,0.0,0.0314,0.030288,0.0,0.00352
4,EasternEurope,SouthernEurope,0.0216,0.0,0.0077,0.0,0.0113,0.009505,0.0,0.06600
5,NorthernEurope,SouthernEurope,0.0774,0.0,0.0685,0.0,0.0502,0.049557,0.0,0.00000


In [9]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt


q1 = BAGs_data['GAP_corrected'].quantile(0.25)
q3 = BAGs_data['GAP_corrected'].quantile(0.75)
iqr = q3 - q1

lower_bound = q1 - 1.0 * iqr
upper_bound = q3 + 1.0 * iqr

BAGs_data_no_outliers = BAGs_data

bins = [0, BAGs_data['GDP_percapita'].median(), float('inf')]  # Umbrales en dólares
labels = ['L_GDP', 'H_GDP']


BAGs_data_no_outliers['Group_GDP'] = pd.cut(BAGs_data_no_outliers['GDP_percapita'], bins=bins, labels=labels)

order = ['L_GDP', 'H_GDP']

regions = ['L_GDP', 'H_GDP']

order = ['L_GDP', 'H_GDP']

regions = ['L_GDP', 'H_GDP']
mann_whitney_results = []

for i in range(len(regions)):
    for j in range(i + 1, len(regions)):
        region1_data = BAGs_data_no_outliers[BAGs_data_no_outliers['Group_GDP'] == regions[i]]['GAP_corrected']
        region2_data = BAGs_data_no_outliers[BAGs_data_no_outliers['Group_GDP'] == regions[j]]['GAP_corrected']


        sub_sample_size = np.min([int(region1_data.shape[0]*0.9), int(region2_data.shape[0]*0.9)])
        region1_data = resample(region1_data, n_samples=sub_sample_size, replace=False, random_state=42)
        region2_data = resample(region2_data, n_samples=sub_sample_size, replace=False, random_state=42)
        
        stat, p_value = stats.mannwhitneyu(region1_data, region2_data, alternative='two-sided')
        
        cliff_d, cliff_ci = cliff_delta(np.array(region1_data), np.array(region2_data))
        print( "cliff delta",cliff_d, cliff_ci)
        
        cliff_d05, cliff_ci05 = cliff_delta05(np.array(region1_data), np.array(region2_data))
        cliff_d10, cliff_ci10 = cliff_delta10(np.array(region1_data), np.array(region2_data))

        r_b = rank_biserial_from_u(np.array(region1_data), np.array(region2_data))
        r_b10 = rank_biserial_from_u10(np.array(region1_data), np.array(region2_data))

        print(f"No parametric cliff_delta: {cliff_d:.3f}")
        print(f"No parametric biseral: {r_b:.3f}")

        p_value_ = permtest(region1_data, region2_data)
        mann_whitney_results.append((regions[i], regions[j], cliff_d, cliff_ci, cliff_d05, cliff_ci05, cliff_d10, r_b10, cliff_ci10, p_value_*len(regions), p_value*len(regions)))


results_list = []
for result in mann_whitney_results:

    result_dict = {
        "Group1": result[0],
        "Group2": result[1],
        "cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[2]),
        "CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[3]),
        "cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[4]),
        "CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[5]),
        "cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[6]),
        "CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[7]),
        "p-value 100000 perm": np.abs(result[8]),
        "p-value mann whitney": np.abs(result[9]),
    }
    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)
display(results_df)


N 29672 14836 14836
cliff delta 0.6098 0.0
N 40826 20413 20413
N 47874 23937 23937
No parametric cliff_delta: 0.610
No parametric biseral: -0.610
Computing permutations...


,Group1,Group2,cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L),CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L),p-value 100000 perm,p-value mann whitney
0,L_GDP,H_GDP,0.6098,0.0,0.4419,0.0,0.3585,0.361031,0.0,0.0


In [10]:
results_df

,Group1,Group2,cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L),CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L),p-value 100000 perm,p-value mann whitney
0,L_GDP,H_GDP,0.6098,0.0,0.4419,0.0,0.3585,0.361031,0.0,0.0


In [11]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt


q1 = BAGs_data['GAP_corrected'].quantile(0.25)
q3 = BAGs_data['GAP_corrected'].quantile(0.75)
iqr = q3 - q1

lower_bound = q1 - 1.0 * iqr
upper_bound = q3 + 1.0 * iqr

BAGs_data_no_outliers = BAGs_data

order = ['L_Ineq', 'H_Ineq']

regions = ['L_Ineq', 'H_Ineq']
mann_whitney_results = []

for i in range(len(regions)):
    for j in range(i + 1, len(regions)):
        region1_data = BAGs_data_no_outliers[BAGs_data_no_outliers['Group_GINI'] == regions[i]]['GAP_corrected']
        region2_data = BAGs_data_no_outliers[BAGs_data_no_outliers['Group_GINI'] == regions[j]]['GAP_corrected']


        sub_sample_size = np.min([int(region1_data.shape[0]*0.9), int(region2_data.shape[0]*0.9)])
        region1_data = resample(region1_data, n_samples=sub_sample_size, replace=False, random_state=42)
        region2_data = resample(region2_data, n_samples=sub_sample_size, replace=False, random_state=42)
        
        stat, p_value = stats.mannwhitneyu(region1_data, region2_data, alternative='two-sided')
        
        cliff_d, cliff_ci = cliff_delta(np.array(region1_data), np.array(region2_data))
        print( "cliff delta",cliff_d, cliff_ci)
        
        cliff_d05, cliff_ci05 = cliff_delta05(np.array(region1_data), np.array(region2_data))
        cliff_d10, cliff_ci10 = cliff_delta10(np.array(region1_data), np.array(region2_data))

        r_b = rank_biserial_from_u(np.array(region1_data), np.array(region2_data))
        r_b10 = rank_biserial_from_u10(np.array(region1_data), np.array(region2_data))

        print(f"No parametric cliff_delta: {cliff_d:.3f}")
        print(f"No parametric biseral: {r_b:.3f}")

        p_value_ = permtest(region1_data, region2_data)
        mann_whitney_results.append((regions[i], regions[j], cliff_d, cliff_ci, cliff_d05, cliff_ci05, cliff_d10, r_b10, cliff_ci10, p_value_*len(regions), p_value*len(regions)))


results_list = []
for result in mann_whitney_results:

    result_dict = {
        "Group1": result[0],
        "Group2": result[1],
        "cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[2]),
        "CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[3]),
        "cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[4]),
        "CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[5]),
        "cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[6]),
        "CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[7]),
        "p-value 100000 perm": np.abs(result[8]),
        "p-value mann whitney": np.abs(result[9]),
    }
    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)
display(results_df)


N 30840 15420 15420
cliff delta -0.5482 0.0
N 42506 21253 21253
N 50260 25130 25130
No parametric cliff_delta: -0.548
No parametric biseral: 0.547
Computing permutations...


,Group1,Group2,cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L),CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L),p-value 100000 perm,p-value mann whitney
0,L_Ineq,H_Ineq,0.5482,0.0,0.3904,0.0,0.3145,0.312506,0.0,0.0


In [12]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt


BAGs_data_no_outliers = BAGs_data

BAGs_data_no_outliers.loc[BAGs_data_no_outliers['country'] == 'Colombia', 'Income'] = 'L'
BAGs_data_no_outliers.loc[BAGs_data_no_outliers['country'] == 'Ecuador', 'Income'] = 'L'
BAGs_data_no_outliers.loc[BAGs_data_no_outliers['country'] == 'Brasil', 'Income'] = 'L'
BAGs_data_no_outliers.loc[BAGs_data_no_outliers['country'] == 'Italy', 'Income'] = 'H'


BAGs_data_no_outliers['Income_v1'] = BAGs_data_no_outliers['Income'] 

BAGs_data_no_outliers['Income_v1'] = BAGs_data_no_outliers['Income_v1'].replace({'UM':'H'})

order = ['H', 'L']

regions = ['H', 'L']
mann_whitney_results = []


    
for i in range(len(regions)):
    for j in range(i + 1, len(regions)):
        region1_data = BAGs_data[BAGs_data['Income'] == regions[i]]['GAP_corrected']
        region2_data = BAGs_data[BAGs_data['Income'] == regions[j]]['GAP_corrected']



        sub_sample_size = np.min([int(region1_data.shape[0]*0.9), int(region2_data.shape[0]*0.9)])
        region1_data = resample(region1_data, n_samples=sub_sample_size, replace=False, random_state=42)
        region2_data = resample(region2_data, n_samples=sub_sample_size, replace=False, random_state=42)
    
        stat, p_value = stats.mannwhitneyu(region1_data, region2_data, alternative='two-sided')
        
        cliff_d, cliff_ci = cliff_delta(np.array(region1_data), np.array(region2_data))
        print( "cliff delta",cliff_d, cliff_ci)
        
        cliff_d05, cliff_ci05 = cliff_delta05(np.array(region1_data), np.array(region2_data))
        cliff_d10, cliff_ci10 = cliff_delta10(np.array(region1_data), np.array(region2_data))

        r_b = rank_biserial_from_u(np.array(region1_data), np.array(region2_data))
        r_b10 = rank_biserial_from_u10(np.array(region1_data), np.array(region2_data))

        print(f"No parametric cliff_delta: {cliff_d:.3f}")
        print(f"No parametric biseral: {r_b:.3f}")

        p_value_ = permtest(region1_data, region2_data)
        mann_whitney_results.append((regions[i], regions[j], cliff_d, cliff_ci, cliff_d05, cliff_ci05, cliff_d10, r_b10, cliff_ci10, p_value_*len(regions), p_value*len(regions)))


results_list = []
for result in mann_whitney_results:

    result_dict = {
        "Group1": result[0],
        "Group2": result[1],
        "cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[2]),
        "CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[3]),
        "cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[4]),
        "CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[5]),
        "cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[6]),
        "CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[7]),
        "p-value 100000 perm": np.abs(result[8]),
        "p-value mann whitney": np.abs(result[9]),
    }
    results_list.append(result_dict)
    
results_df = pd.DataFrame(results_list)
display(results_df)


N 25394 12697 12697
cliff delta -0.5444 0.0
N 34244 17122 17122
N 40028 20014 20014
No parametric cliff_delta: -0.544
No parametric biseral: 0.539
Computing permutations...


,Group1,Group2,cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L),CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L),p-value 100000 perm,p-value mann whitney
0,H,L,0.5444,0.0,0.3779,0.0,0.3067,0.300594,0.0,0.0
